# Linear Regression & GLMs — Student Lab

Complete all TODOs. Avoid sklearn for core parts.

In [1]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic Dataset (with collinearity)
We generate data where features can be highly correlated to motivate ridge.

In [2]:
def make_regression(n=250, d=5, noise=0.5, collinear=True):
    X = rng.standard_normal((n, d))
    if collinear and d >= 2:
        X[:, 1] = X[:, 0] + 0.05 * rng.standard_normal(n)
    w_true = rng.standard_normal(d)
    y = X @ w_true + noise * rng.standard_normal(n)
    return X, y, w_true

X, y, w_true = make_regression()
n, d = X.shape
check('shapes', y.shape == (n,))
print('corr(x0,x1)=', np.corrcoef(X[:,0], X[:,1])[0,1])

OK: shapes
corr(x0,x1)= 0.9984746876366171


## Section 1 — OLS Closed Form

### Task 1.1: Closed-form w_hat using solve

# TODO: compute w_hat using solve on (X^T X) w = X^T y
# HINT: `XtX = X.T@X`, `Xty = X.T@y`, `np.linalg.solve(XtX, Xty)`

**Checkpoint:** Why is explicit inverse discouraged?

explicit inverse is more unstable and computationally expensive.

In [ ]:
# TODO
XtX = X.T @ X
Xty = X.T @ y
w_hat = np.linalg.solve(XtX, Xty) 
 
check('w_shape', w_hat.shape == (d,))

OK: w_shape


### Task 1.2: Evaluate fit + residuals
Compute:
- predictions y_pred
- MSE
- residual mean and std

**Interview Angle:** What does a structured residual pattern imply (e.g., nonlinearity)?

it imples current model is not exactly suitable. it failed to capture a pattern. we should try others models. for example, if the current model is a linear model. we should try polynomial model or see if all features are included in linear model or look for any corelation errors.

In [17]:
# TODO
y_pred = np.dot(X, w_hat) # predicting y from X and w_hat : y_pred = X * w_hat
mse = float(np.mean((y - y_pred) ** 2)) # mean squared error between y and y_pred
resid = y_pred - y # residuals between y_pred and y

print('mse', mse, 'resid_mean', resid.mean(), 'resid_std', resid.std())
check('finite', np.isfinite(mse))

mse 0.260799382964824 resid_mean 0.022970458235497877 resid_std 0.5101683457578245
OK: finite


## Section 2 — Gradient Descent

### Task 2.1: Implement MSE loss + gradient

Loss = mean((Xw-y)^2), grad = (2/n) X^T(Xw-y)

# TODO: implement `mse_loss_and_grad`

**FAANG gotcha:** shapes and constants.

In [ ]:
def mse_loss_and_grad(X, y, w):
    # TODO
    X = np.asarray(X, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    w = np.asarray(w, dtype=np.float64)
    
    r = X @ w - y
    """
    in class we used (1/n) sum r_i^2 for loss function formula. Here I used (1/2n) sum r_i^2.
    because many standard reference books were using this form. I hoope it's ok.
    gradient formula is adusted accordingly.
    """
    loss = 0.5 * np.mean(r ** 2) # mean squared error loss
    grad = (X.T @ r) / X.shape[0] # gradient of loss
    return loss, grad

w0 = np.zeros(d)
loss0, g0 = mse_loss_and_grad(X, y, w0)
check('grad_shape', g0.shape == (d,))
check('finite_loss', np.isfinite(loss0))

OK: grad_shape
OK: finite_loss


### Task 2.2: Train with GD + compare to closed-form

# TODO: implement a simple GD loop, track loss, and compare final weights to w_hat.

**Checkpoint:** How does feature scaling affect GD?

feature scaling changes the value of corrospding weights. when we adjust the weights with GD, using feature scaling makes GD stable and smooth by avoiding zig-zag pattern.

In [ ]:
def train_gd(X, y, lr=0.05, steps=500):
    # TODO
    X = np.asarray(X, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    
    w = np.zeros(X.shape[1]) # initializing weights to zero as starting point
    losses = [] # to store loss values
    for i in range(steps): #steps is number of iterations
        loss, grad = mse_loss_and_grad(X, y, w) # computing loss and gradient
        w = w - lr * grad # updating weights
        losses.append(loss)
    return w, losses

w_gd, losses = train_gd(X, y, lr=0.05, steps=500)
print('final_loss', losses[-1])
print('||w_gd-w_hat||', np.linalg.norm(w_gd - w_hat))
check('loss_decreases', losses[-1] <= losses[0])

final_loss 0.13150579862647271
||w_gd-w_hat|| 1.3164915910450554
OK: loss_decreases


## Section 3 — Ridge Regression (L2)

### Task 3.1: Ridge closed-form
w = (X^T X + λI)^{-1} X^T y

# TODO: implement ridge_solve

**Interview Angle:** Why does ridge help under collinearity?

Ridge regression helps in recuding high varience by altering weights(by adding additional cost from weights). when the features are linearly dependent it shrinks them thus helping with collinearity.

In [ ]:
def ridge_solve(X, y, lam):
    # TODO
    X = np.asarray(X, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    lam = lam if lam > 0 else 0.0 # ensure lambda is non-negative
    d = X.shape[1] # number of features
    return np.linalg.solve(X.T @ X + lam * np.eye(d), X.T @ y) # ridge regression formula, lambda = lam = 1.0 here.
 
w_ridge = ridge_solve(X, y, lam=1.0)
check('ridge_shape', w_ridge.shape == (d,))

OK: ridge_shape


### Task 3.2: Bias/variance demo with train/test split

# TODO: split into train/test and compare MSE for multiple lambdas.

**Checkpoint:** why can test error improve even when train error worsens?

Because the model is generalizing (Not overfitting to training data) which makes the train error worse. since its generalizing the test data fits better, hence test error is lower.

In [16]:
# TODO
idx = rng.permutation(n)
train = idx[: int(0.7*n)]
test = idx[int(0.7*n):]
Xtr, ytr = X[train], y[train]
Xte, yte = X[test], y[test]

lams = [0.0, 0.1, 1.0, 10.0]
results = []
for lam in lams:
    w = ridge_solve(Xtr, ytr, lam=lam) if lam > 0 else np.linalg.solve(Xtr.T@Xtr, Xtr.T@ytr)
    tr_mse = np.mean((Xtr@w - ytr)**2)
    te_mse = np.mean((Xte@w - yte)**2)
    results.append((lam, tr_mse, te_mse))
print('lam, train_mse, test_mse')
for r in results:
    print(r)

lam, train_mse, test_mse
(0.0, 0.2673763268904879, 0.2500584551029355)
(0.1, 0.26761204346193085, 0.25016353854476864)
(1.0, 0.26907830138935623, 0.25188423277193805)
(10.0, 0.2755994368557424, 0.26284506240445366)


## Section 4 — GLM Intuition

### Task 4.1: Match tasks to (distribution, link)
Fill in a table for:
- regression
- binary classification
- count prediction

**Explain:** what changes when you go from OLS to a GLM?

GLM can handle non - linear(like polynomial) and non - gaussian (like classification) realtions while OLS cannot be used for them.

| Problem | Target type | Distribution | Link | Loss |
|---|---|---|---|---|
| House price | continuous | Guassian | identity | MSE |
| Fraud | binary | Bernouli | logit | BCE / log loss |
| Clicks per user | count | poisson | log | poisson devience |


---
## Submission Checklist
- All TODOs completed
- Train/test results shown for ridge
- Short answers to checkpoint questions
